# Launch and compare multiple batch jobs

In [8]:
%pip install --upgrade boto3 botocore pydantic jsonlines

Note: you may need to restart the kernel to use updated packages.


### Rework the dataset you want to use batch inference for

In [46]:
import pandas as pd

ragtestdata = 'hf://datasets/neural-bridge/rag-dataset-12000/data/test-00000-of-00001-af2a9f454ad1b8a3.parquet'
df = pd.read_parquet(ragtestdata)
df_new = pd.DataFrame()
df_new['text'] = df['context']+ "\n" + df['question']

# Insert manually or use base config below in next cell
# df_new['temperature']=[0.0]*len(df_new)
# df_new['max_tokens']=[512]*len(df_new)
# df_new['record_id'] = 'REC' + df.index.astype(str).str.zfill(8)
df_new

,text
0,"HOUSTON (Jan. 23, 2018) – Fabien Gabel, music director of the Quebec Symphony Orchestra, returns to Houston to lead the Houston Symphony in Ravel’s Daphnis and Chloé on Feb. 2 and 3 at 8 p.m. and Feb. 4 at 2:30 p.m. in Jones Hall.\nRecognized internationally as one of the stars of the new generation, Fabien Gabel is a regular guest of the Houston Symphony and an audience favorite. Known for conducting music with French influences, Gabel leads the Symphony in a program of French and American ..."
1,"Port Harcourt – The Rivers police command on Wednesday arraigned 13 suspects before a Port Harcourt Magistrates’ Court charged with conspiracy and murder of four students of the University of Port Harcourt.\nThe suspects were arraigned on a five-count charge of murder, felony, conspiracy, lynching, and burning.\nProsecuting police Adiari Idafi told the court, presided over by Magistrate Emma Woke, that the accused on Oct. 5, allegedly murdered Chiadika Lordson, Ugonna Kelechi Obusor, Mike Ll..."
2,"Channels\nMusic\nStyle\nPop Culture\nSports\nSneakers\nLife\nVideos\nMore from Complex\nGO90\nRated Red\nFirst We Feast\nPigeons & Planes\nCollider\nSole Collector\nGreen Label\nPaul Wall\nMusic\nHere's Everything You Need To Know About Paul Wall And Baby Bash's Drug Arrest\nBy Omar Burgess\nDec 25, 2016\nMusic\nPremiere: Stream Paul Wall's 'Houston Oiler' Album f/ Chamillionaire, Slim Thug, Z-Ro And More\nBy Edwin Ortiz\nOct 20, 2016\nMusic\nPaul Wall Wants To Give Free Gold Grills To All U..."
3,"The crazy level at which African countries import things was reiterated by the president of Zimbabwe, President Emmerson Mnangagwa, while addressing the delegates at the Southern African Confederation of Agricultural Unions (SACAU) annual meeting in Victoria Falls. Hear him: ‘it is regrettable that African countries spend between $30 billion and $50billion annually on imports of agricultural products, instead of developing the productive capacities necessary for trade’.\nAccording to the Uni..."
4,"CHI 2010 Workshop May 7 or 8, 2011 (final date to be announced)\nCall for Participation\nLarge interactive displays are now common in public urban life. Museums, libraries, public plazas, and architectural facades already take advantage of interactive technologies for visual and interactive information presentation. Researchers and practitioners from such varied disciplines as art, architecture, design, HCI, and media theory have started to explore the potential and impact of large display i..."
...,...
2395,"Fuzzy's Ultra Premium Vodka\nThe Myth, The Man, The Golf Legend, Fuzzy Zoeller is known around the world for his golfing success, charismatic character and outgoing personality! His credits include winning ten times on the PGA Tour, including the 1979 Masters and 1984 U.S. Open along with the 1985 and 1986 Skins Game, just to name just a few.\nFuzzy’s commitment to golf goes beyond playing the game, as he is also well known for his golf course design, and designing nineteen courses, 3 of whi..."
2396,"Swedish Grand Prix rider Malin Nilsson got married to her partner, German Steffen Wahlkamp, on Saturday 2 June 2018. ""We signed the paipers today with our closest friends and family and next year we will celebrate big,"" said Nilsson about her civil marriage. Her close friends, Australians Briana Burgess, Lyndal Oatley and employer Patrik Kittel, were amongst the guests.\nThe 80-year old Swedish Grand Prix dressage rider Ulla Hakanson continues to set records! By competing at the 2018 Swedish..."
2397,"The Cracchiolo Law Library of the James E. Rogers College of Law and the University of Arizona School of Information offer a Fellowship in Law Librarianship for lawyers seeking to become law librarians. The Fellowship program began in 2001 and our graduates have been placed in law school, university, court, and law firm libraries across the country.\nFellows work 20 hours a week in the law library in a variety of professional roles while at

### Submit batch jobs for multiple models

In [2]:
import boto3

bedrock = boto3.client('bedrock')
from batchhelper import *
import uuid
config = BaseGenerationConfig(
    temperature=0.0,
    top_p=0.99,
    max_tokens=512,
    stop_sequences=[],
    top_k=250,
    system="You are a helpful assistant. Please use the context to answer the question that follows."
)

model_list = {"anthropic.claude-3-sonnet-20240229-v1:0":ModelType.CLAUDE,
              "anthropic.claude-3-haiku-20240307-v1:0":ModelType.CLAUDE,
              "amazon.nova-micro-v1:0":ModelType.NOVA,
              "amazon.nova-lite-v1:0":ModelType.NOVA,
              "amazon.nova-pro-v1:0":ModelType.NOVA,
              "us.meta.llama3-3-70b-instruct-v1:0":ModelType.LLAMA
             }


for model in model_list.keys():

# Convert DataFrame to JSONL
    dataframe_to_jsonl(
        df=df_new,
        model_type=model_list[model],
        output_file=f'batch_requests_{model_list[model].value}.jsonl',
        text_column='text',
        record_id_column='custom_id',
        base_config=config
    )

    s3path = f"batch_requests_{model_list[model].value}.jsonl"
    print(f"Uploading to s3://subshreyevals/{s3path}")
    !aws s3 cp {s3path} s3://subshreyevals/{s3path}

    inputDataConfig=({
    "s3InputDataConfig": {
        "s3Uri": f"s3://subshreyevals/{s3path}"
        }
        })

    outputDataConfig=({
        "s3OutputDataConfig": {
            "s3Uri": "s3://subshreyevals/"
        }
    })

    response=bedrock.create_model_invocation_job(
        roleArn="arn:aws:iam::716845917484:role/service-role/novabatch",
        modelId=model,
        jobName=f"{model_list[model].value}-eval-{str(uuid.uuid4())[:8]}-new",
        inputDataConfig=inputDataConfig,
        outputDataConfig=outputDataConfig
    )

    jobArn = response.get('jobArn')
    print(bedrock.get_model_invocation_job(jobIdentifier=jobArn)['status'])
    print("---")
    


Uploading to s3://subshreyevals/batch_requests_claude.jsonl
upload: ./batch_requests_claude.jsonl to s3://subshreyevals/batch_requests_claude.jsonl
Submitted
---
Uploading to s3://subshreyevals/batch_requests_claude.jsonl
upload: ./batch_requests_claude.jsonl to s3://subshreyevals/batch_requests_claude.jsonl
Submitted
---
Uploading to s3://subshreyevals/batch_requests_nova.jsonl
upload: ./batch_requests_nova.jsonl to s3://subshreyevals/batch_requests_nova.jsonl
Submitted
---
Uploading to s3://subshreyevals/batch_requests_nova.jsonl
upload: ./batch_requests_nova.jsonl to s3://subshreyevals/batch_requests_nova.jsonl
Submitted
---
Uploading to s3://subshreyevals/batch_requests_nova.jsonl
upload: ./batch_requests_nova.jsonl to s3://subshreyevals/batch_requests_nova.jsonl
Submitted
---
Uploading to s3://subshreyevals/batch_requests_llama.jsonl
upload: ./batch_requests_llama.jsonl to s3://subshreyevals/batch_requests_llama.jsonl
Submitted
---


### Monitor and filter jobs

In [53]:
import boto3
bedrock = boto3.client('bedrock')
import pandas as pd
df_jobs = pd.DataFrame(bedrock.list_model_invocation_jobs()['invocationJobSummaries'])
df_jobs['jobtime'] = df_jobs['endTime'] - df_jobs['submitTime']
df_jobs[~df_jobs['status'].isin(['Failed', 'Stopped'])]

,jobArn,jobName,modelId,roleArn,status,submitTime,lastModifiedTime,endTime,inputDataConfig,outputDataConfig,timeoutDurationInHours,jobtime
1,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/6dy9j0cby1hy,nova-eval-1cb4b733-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-16 19:42:37.583000+00:00,2025-01-17 01:10:57.081000+00:00,2025-01-17 01:10:56.649000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_nova.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 05:28:19.066000
2,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/w1lxg733nb4p,nova-eval-6b3551f6-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-16 19:42:35.742000+00:00,2025-01-16 21:06:06.321000+00:00,2025-01-16 21:06:05.042000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_nova.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 01:23:29.300000
3,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/8o8n3la7sh9f,nova-eval-c5a51d6b-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-16 19:42:33.955000+00:00,2025-01-16 20:01:57.897000+00:00,2025-01-16 20:01:57.407000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_nova.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 00:19:23.452000
4,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/465vp0bf0oir,claude-eval-6bd5d4d6-new,arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-16 19:42:32.024000+00:00,2025-01-16 20:32:00.801000+00:00,2025-01-16 20:32:00.351000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_claude.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 00:49:28.327000
5,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/u1es5e9052mg,claude-eval-0f8dd3e8-new,arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Expired,2025-01-16 19:42:29.930000+00:00,2025-01-17 19:42:54.859000+00:00,2025-01-16 19:46:15.699000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_claude.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 00:03:45.769000
19,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/e83i3eq809yv,nova-eval-ea181750,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-15 20:31:20.693000+00:00,2025-01-15 22:18:53.672000+00:00,2025-01-15 22:18:53.179000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_nova.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataConfig': {'s3Uri': 's3://subshreyevals/', 's3BucketOwner': '716845917484'}}",24,0 days 01:47:32.486000
20,arn:aws:bedrock:us-east-1:716845917484:model-invocation-job/w1574to5tiet,nova-eval-06499302,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0,arn:aws:iam::716845917484:role/service-role/novabatch,Completed,2025-01-15 20:31:18.753000+00:00,2025-01-15 23:58:49.800000+00:00,2025-01-15 23:58:49.370000+00:00,"{'s3InputDataConfig': {'s3Uri': 's3://subshreyevals/batch_requests_nova.jsonl', 's3BucketOwner': '716845917484'}}","{'s3OutputDataCon

In [54]:
pd.set_option("max_colwidth", 500)
df_jobs.loc[df_jobs['status'].isin(['Completed']),['jobName','modelId','jobtime']]

,jobName,modelId,jobtime
1,nova-eval-1cb4b733-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0,0 days 05:28:19.066000
2,nova-eval-6b3551f6-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0,0 days 01:23:29.300000
3,nova-eval-c5a51d6b-new,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0,0 days 00:19:23.452000
4,claude-eval-6bd5d4d6-new,arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0,0 days 00:49:28.327000
19,nova-eval-ea181750,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0,0 days 01:47:32.486000
20,nova-eval-06499302,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0,0 days 03:27:30.617000
21,nova-eval-e858b159,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0,0 days 00:22:53.303000
22,claude-eval-f940c407,arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0,0 days 02:45:48.566000
25,nova-eval-937649cf,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0,0 days 01:42:16.764000
27,nova-eval-cb42ea62,arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:0,0 days 00:22:02.345000


### Download and view final output of a single job

In [55]:
import ipywidgets as widgets
from IPython.display import display

completed_jobs = list(df_jobs[df_jobs['status'].isin(['Completed'])].jobArn)
completed_jobs_names = list(df_jobs[df_jobs['status'].isin(['Completed'])].jobName)
jobids = [c.split('/')[-1] for c in completed_jobs]

job_mapping = dict(zip(completed_jobs_names, jobids))
find_name = lambda value:list(job_mapping.keys())[list(job_mapping.values()).index(value)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=completed_jobs_names,
    value=completed_jobs_names[0],
    description='Select an job:'
)

# Display the dropdown
display(dropdown)



Dropdown(description='Select an job:', options=('nova-eval-1cb4b733-new', 'nova-eval-6b3551f6-new', 'nova-eval…

In [60]:
import json
import jsonlines
import glob
jobid = job_mapping[dropdown.value]

!aws s3 cp --recursive s3://subshreyevals/{jobid}/ ./{jobid}/


output_text = []
with jsonlines.open(glob.glob(f'{jobid}/*.jsonl.out')[0]) as reader:
    for data in reader:
        # this works for Claude and Nova
        try:
            if 'nova' in find_name(jobid):
                output_text.append(data['modelOutput']['output']['message']['content'][0]['text'])
            elif 'claude' in find_name(jobid):
                output_text.append(data['modelOutput']['content'][0]['text'])
                
        except:
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data)
            output_text.append("Error")
            break

df_output = df_new.copy()
df_output['output'] = output_text

print(json.load(open(f'{jobid}/manifest.json.out','r')))

df_output

download: s3://subshreyevals/6dy9j0cby1hy/manifest.json.out to 6dy9j0cby1hy/manifest.json.out
download: s3://subshreyevals/6dy9j0cby1hy/batch_requests_nova.jsonl.out to 6dy9j0cby1hy/batch_requests_nova.jsonl.out
{'totalRecordCount': 2400, 'processedRecordCount': 2400, 'successRecordCount': 2400, 'errorRecordCount': 0, 'inputTokenCount': 1951382, 'outputTokenCount': 405633}


,text,nova-eval-6b3551f6-new,nova-eval-c5a51d6b-new,claude-eval-6bd5d4d6-new,output
0,"HOUSTON (Jan. 23, 2018) – Fabien Gabel, music director of the Quebec Symphony Orchestra, returns to Houston to lead the Houston Symphony in Ravel’s Daphnis and Chloé on Feb. 2 and 3 at 8 p.m. and Feb. 4 at 2:30 p.m. in Jones Hall.\nRecognized internationally as one of the stars of the new generation, Fabien Gabel is a regular guest of the Houston Symphony and an audience favorite. Known for conducting music with French influences, Gabel leads the Symphony in a program of French and American ...",The music director of the Quebec Symphony Orchestra is Fabien Gabel.,Fabien Gabel is the music director of the Quebec Symphony Orchestra.,"According to the context, the music director of the Quebec Symphony Orchestra is Fabien Gabel. The passage states that ""Fabien Gabel, music director of the Quebec Symphony Orchestra, returns to Houston to lead the Houston Symphony in Ravel's Daphnis and Chloé on Feb. 2 and 3 at 8 p.m. and Feb. 4 at 2:30 p.m. in Jones Hall.""",The music director of the Quebec Symphony Orchestra is Fabien Gabel.
1,"Port Harcourt – The Rivers police command on Wednesday arraigned 13 suspects before a Port Harcourt Magistrates’ Court charged with conspiracy and murder of four students of the University of Port Harcourt.\nThe suspects were arraigned on a five-count charge of murder, felony, conspiracy, lynching, and burning.\nProsecuting police Adiari Idafi told the court, presided over by Magistrate Emma Woke, that the accused on Oct. 5, allegedly murdered Chiadika Lordson, Ugonna Kelechi Obusor, Mike Ll...",The four students of the University of Port Harcourt that were allegedly murdered are:\n\n1. Chiadika Lordson\n2. Ugonna Kelechi Obusor\n3. Mike Lloyd Toku\n4. Tekena Elkanah,The four students of the University of Port Harcourt who were allegedly murdered are:\n\n1. Chiadika Lordson\n2. Ugonna Kelechi Obusor\n3. Mike Lloyd Toku\n4. Tekena Elkanah,"According to the article, the four students of the University of Port Harcourt who were allegedly murdered are:\n\n1. Chiadika Lordson\n2. Ugonna Kelechi Obusor\n3. Mike Lloyd Toku\n4. Tekena Elkanah\n\nThe article states that the 13 suspects were arraigned and charged with the conspiracy and murder of these four students.",The four students of the University of Port Harcourt who were allegedly murdered are:\n\n1. Chiadika Lordson\n2. Ugonna Kelechi Obusor\n3. Mike Lloyd Toku\n4. Tekena Elkanah
2,"Channels\nMusic\nStyle\nPop Culture\nSports\nSneakers\nLife\nVideos\nMore from Complex\nGO90\nRated Red\nFirst We Feast\nPigeons & Planes\nCollider\nSole Collector\nGreen Label\nPaul Wall\nMusic\nHere's Everything You Need To Know About Paul Wall And Baby Bash's Drug Arrest\nBy Omar Burgess\nDec 25, 2016\nMusic\nPremiere: Stream Paul Wall's 'Houston Oiler' Album f/ Chamillionaire, Slim Thug, Z-Ro And More\nBy Edwin Ortiz\nOct 20, 2016\nMusic\nPaul Wall Wants To Give Free Gold Grills To All U...","Paul Wall offered to give free gold grills to all U.S. Olympic Medalists. This was mentioned in the article titled ""Paul Wall Wants To Give Free Gold Grills To All U.S. Olympic Medalists"" by Joshua Espinoza, published on August 22, 2016.","Paul Wall offered to give free gold grills to all U.S. Olympic medalists. This information is mentioned in the article titled ""Paul Wall Wants To Give Free Gold Grills To All U.S. Olympic Medalists"" by Joshua Espinoza, published on August 22, 2016.","According to the context, Paul Wall offered to give free gold grills to all U.S. Olympic medalists. Specifically, the article states:\n\n""Paul Wall Wants To Give Free Gold Grills To All U.S. Olympic Medalists""\n\nSo Paul Wall wanted to provide free custom gold grills (teeth jewelry) to any U.S. athletes who won medals at the Olympics.","Paul Wall offered to give free gold grills to all U.S. Olympic Medalists. This was mentioned in the article titled ""Paul Wall Wants To Give Free Gold Grills To All U.S. Olympic Me

### Compare outputs of multiple jobs

In [61]:
import ipywidgets as widgets

# Create the multiselect widget
multiselect = widgets.SelectMultiple(
    options=completed_jobs_names,
    value=[completed_jobs_names[0]],
    description='Select multiple jobs to compare:',
    disabled=False
)

# Display the widget
display(multiselect)

SelectMultiple(description='Select multiple jobs to compare:', index=(0,), options=('nova-eval-1cb4b733-new', …

In [62]:
# Get the selected values
print("Comparing...")
selected_jobs = multiselect.value
print(selected_jobs)

Comparing...
('nova-eval-1cb4b733-new', 'nova-eval-6b3551f6-new', 'nova-eval-c5a51d6b-new', 'claude-eval-6bd5d4d6-new')


In [63]:
output_text = {}
for jobn in list(selected_jobs):
    output_text[jobn]=[]    
    path = glob.glob(job_mapping[jobn]+'/*.jsonl.out')[0]
    with jsonlines.open(path) as reader:
        for data in reader:
            # this works for Claude and Nova
            try:
                if 'nova' in jobn:
                    output_text[jobn].append(data['modelOutput']['output']['message']['content'][0]['text'])
                elif 'claude' in jobn:
                    output_text[jobn].append(data['modelOutput']['content'][0]['text'])
            except Exception as e:
                print(e)
                break
                output_text[jobn].append(str(e))
    
    df_new[jobn] = output_text[jobn]

df_new.sample(10)

,text,nova-eval-6b3551f6-new,nova-eval-c5a51d6b-new,claude-eval-6bd5d4d6-new,nova-eval-1cb4b733-new
1570,Hi guys! Updating my blog for today! Just had a swim earlier and it’s been a long time since I had one. Enjoying our new home here in Kuala Lumpur Malaysia and everything is so good since we arrived here and totally enjoying it. We’re trying to plan a lot of exploration in this country and so far we have all the good foods we can get just a stone away from Suria KLCC. It’s been raining here ever since we moved in to the new house but not a big deal at all. It’s making the weather more friend...,"The author is currently residing in Kuala Lumpur, Malaysia.","The author is currently residing in Kuala Lumpur, Malaysia. They mention enjoying their new home in the city and exploring the area.","Based on the context provided, the author is currently residing in Kuala Lumpur, Malaysia. Some key details that indicate this:\n\n- The author mentions ""our new home here in Kuala Lumpur Malaysia"" and that they are ""enjoying it"" since arriving there.\n\n- The author talks about exploring the country and enjoying the good food that is just ""a stone away from Suria KLCC"" (a major shopping mall in Kuala Lumpur).\n\n- The author mentions it has been raining since they moved into their ""new hous...","The author is currently residing in Kuala Lumpur, Malaysia. They mention enjoying their new home there and the various amenities and experiences they are having, such as the good food available near Suria KLCC and the pleasant weather despite the rain."
2100,"Print | Save | Directions\nAboutAclpi is located at the address 202 E Mcdowell Rd in Phoenix, Arizona 85004.\nFor more information go to aclpi.org\nFor maps and directions to Aclpi view the map to the right. For reviews of Aclpi see below.\nCONTACT INFORMATION\nCATEGORIES:\nPeople Also Viewed\nSaint Pamela G Attorney\n846 N 6th Ave Phoenix, Arizona 85003\nFERRAGUT ASSOCIATES\n2 N Central Ave 725 Ste Phoenix, Arizona 85004\nSchultz Christopher M Attorney\n2901 N Central Ave Phoenix, Arizona 8...","Aclpi is located at the address 202 E Mcdowell Rd in Phoenix, Arizona 85004.","Aclpi is located at 202 E Mcdowell Rd in Phoenix, Arizona 85004. For maps and directions, you can view the map to the right on the website.","According to the context provided, Aclpi is located at the address 202 E Mcdowell Rd in Phoenix, Arizona 85004.","Aclpi is located at the address 202 E McDowell Rd in Phoenix, Arizona 85004."
1178,"- Lanchas black jack venezuela venezuel of Qtableview slots metal melting furnaces for more than 30 years. Contact Saunders today and well lanchas black jack venezuela black jack mit jasskarten select the metal lanchas black jack venezuela furnace you need lanchas black jack venezuela get you the best price available!, Hill-Rom-Advance-Bed-Service-manual (1). pdf - Jaci ebook download as PDF File (. pdf), Text File (. txt) or read book online for free. Certainteed bolt slot 06, 2016nbsp;0183...",The game based on poker hand rankings mentioned in the context is **Chinese poker**.,"The game based on poker hand rankings mentioned in the context is **Chinese poker**. The context notes that Chinese poker is a card game where players need a basic knowledge of poker hand rankings to get started, and it involves a significant element of luck, giving beginners a chance to win against experienced opponents.","Based on the context provided, the game mentioned that is based on poker hand rankings is Chinese poker. The passage states:\n\n""Chinese poker is a card game based on poker hand rankings. The rules are simple-only a basic knowledge of poker hand rankings is needed to get started.""\n\nSo Chinese poker is the game described that uses poker hand rankings as the basis for the game.","The game based on poker hand rankings mentioned in the context is Chinese poker. Chinese poker, also known as Open Face Chinese Poker (OFCP), is a variant of poker where players are dealt 13 cards each and mus